# More GroupBy Functionality

### Objectives
After this lesson you should be able to...
+ Know how to use the primary groupby methods **`agg`**, **`filter`**, **`transform`** and **`apply`**
+ Know the differences between **`agg`**, **`filter`**, **`transform`** and **`apply`**
+ Know what object is implicitly passed to **`agg`**, **`filter`**, **`transform`** and **`apply`**

### Prepare for this lesson by
+ Reading the rest of the [split apply combine](http://pandas.pydata.org/pandas-docs/stable/groupby.html#transformation) documentation from Transformation until the end

# Difficulty Building Multi-line Custom Aggregation Functions
It can be quite difficult to build custom aggregation functions, because we can't easily "see" what is going on during each function call. Many times it is possible to get your desired result without them and if this is indeed possible, avoid the custom aggregation function.

Let's build a multi-line custom aggregation function that finds the [geometric mean][1]. This involves finding the product of all Series values and then taking the **nth** root of this value where **n** is the number of observations. 

[1]:https://en.wikipedia.org/wiki/Geometric_mean

In [1]:
def geomean(s):
    prod = s.product()
    power = 1 / len(s)
    return pow(prod, power)

### Take geometric mean of `UGDS` column

In [2]:
import pandas as pd
import numpy as np
college = pd.read_csv('../../data/college.csv')
college.head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


In [3]:
college.groupby('STABBR').agg({'UGDS': geomean}).head(15)

,UGDS
STABBR,
AK,615.160954
AL,550.410442
AR,345.606285
AS,1276.000000
AZ,inf
CA,0.000000
CO,0.000000
CT,0.000000
DC,109.751053


## Why are we getting `inf` and `0`?
We are returned some values that we may not have expected. How did infinity get returned (**`inf`**)? We are probably getting **`0`** because some of the colleges have 0 population. It's not obvious on what we should do to inspect what the problem is.

## Print out values in the function
One helpful trick is to use the **`print`** function to output values that you want to understand during the aggregation. To start off simple, let's print out the first three values of the Series for each group.

In [4]:
def geomean(s):
    print(s.head(3), end='\n\n')
    prod = s.product()
    power = 1 / len(s)
    return pow(prod, power)

In [5]:
df = college.groupby(['STABBR', 'RELAFFIL']).agg({'UGDS': geomean})

60    12865.0
62     5536.0
63     1428.0
Name: UGDS, dtype: float64

61       27.0
64      275.0
5417     68.0
Name: UGDS, dtype: float64

0     4206.0
1    11383.0
3     5451.0
Name: UGDS, dtype: float64

2      291.0
10    1180.0
12     322.0
Name: UGDS, dtype: float64

128    7942.0
129     711.0
130      38.0
Name: UGDS, dtype: float64

131    896.0
134    690.0
144    802.0
Name: UGDS, dtype: float64

4138    1276.0
Name: UGDS, dtype: float64

69     83.0
71    126.0
72      1.0
Name: UGDS, dtype: float64

68    4102.0
70      25.0
73      89.0
Name: UGDS, dtype: float64

192    9885.0
193     500.0
194       NaN
Name: UGDS, dtype: float64

200       NaN
210    5918.0
214     261.0
Name: UGDS, dtype: float64

565      75.0
566    2110.0
567    3683.0
Name: UGDS, dtype: float64

575    3657.0
589       0.0
592       NaN
Name: UGDS, dtype: float64

629     137.0
630    1316.0
631     493.0
Name: UGDS, dtype: float64

628    1249.0
645    3841.0
652      58.0
Name: UGDS, dtype: floa

### Print out something more useful
As you can see, the **`print`** function may be used to inspect and debug the logic of a customized aggregation function. The first three Series values aren't particularly useful in this example.

#### NumPy `where` function
The NumPy **`where`** function accepts an array of values. It returns the integer location of all the True values. Take the following 4-element list. It returns the locations 0 and 3.

In [6]:
np.where([True, False, False, True])

(array([0, 3]),)

It returns the result inside of a one-element tuple. You must select the first element of the tuple to get the array:

In [7]:
idx_true = np.where([True, False, False, True])
idx_true[0]

array([0, 3])

## Use `np.where` to find the location of `inf`
Going back to our geometric mean example, we can use the **`np.where`** function to find the integer locations of infinite values. Pandas has no default way to find infinite values. We must use NumPy again which has the **`np.inf`** object.

## Taking Cumulative Product
Inside the function, we can take the cumulative product to determine exactly where the product became infinite. The following will print out the section of the cumulative product Series right before an infinite value is found.

In [8]:
def geomean(s):
    cumprod = s.cumprod()
    inf_idx = np.where(cumprod == np.inf)[0]
    if len(inf_idx) > 0:
        idx = inf_idx[0]
        print(cumprod.iloc[idx - 1: idx + 2])
    prod = s.product()
    power = 1 / len(s)
    return pow(prod, power)

In [9]:
college.groupby(['STABBR', 'RELAFFIL']).agg({'UGDS': geomean})

6993    1.447969e+308
7023              inf
7027              inf
Name: UGDS, dtype: float64
825    5.110238e+307
826              inf
827              inf
Name: UGDS, dtype: float64
5878    1.892396e+306
6059              inf
6083              inf
Name: UGDS, dtype: float64
1123    3.610109e+306
1124              inf
1125              inf
Name: UGDS, dtype: float64
1750    1.362749e+307
1751              inf
1752              inf
Name: UGDS, dtype: float64
5439    2.963452e+307
5449              inf
5506              inf
Name: UGDS, dtype: float64
5953    6.842536e+306
5954              inf
6018              inf
Name: UGDS, dtype: float64
2769    4.245768e+306
2770              inf
2771              inf
Name: UGDS, dtype: float64
5793    3.997530e+307
5814              inf
5835              inf
Name: UGDS, dtype: float64
2447    2.907034e+307
2449              inf
2450              inf
Name: UGDS, dtype: float64
2939    8.646545e+307
2940              inf
2941              inf
Name: U

UGDS
STABBR RELAFFIL             
AK     0         1477.495053
       1           79.628486
AL     0          954.573936
       1          105.515908
AR     0          446.853645
       1          130.931249
AS     0         1276.000000
AZ     0                 inf
       1          100.743883
CA     0            0.000000
       1           18.478397
CO     0            0.000000
       1            0.000000
CT     0          352.804482
       1            0.000000
DC     0          185.772229
       1           40.612656
DE     0          400.738032
       1         3224.844533
FL     0                 inf
       1           13.757604
FM     0         2344.000000
GA     0                 inf
       1          110.819478
GU     0         2692.595402
       1           65.000000
HI     0          391.565880
       1          909.473580
IA     0          350.639142
       1          407.235364
...                      ...
PA     0                 NaN
       1           97.241603
PR     0                 inf
       1          157.583273
PW     0          602.000000
RI     0          489.641780
       1          174.422057
SC     0          465.584028
       1          128.564705
SD     0          589.808574
       1          146.796112
TN     0                 inf
       1          172.427160
TX     0                 inf
       1           47.886467
UT     0          269.133271
       1           35.905112
VA     0                 inf
       1           36.310021
VI     0         1971.000000
       1            1.000000
VT     0          230.100063
       1           48.251769
WA     0          470.976524
       1          101.881837
WI     0          482.391521
       1          189.531313
WV     0          259.726381
       1          224.203223
WY     0         1039.607349

[112 rows x 1 columns]

## Numeric Overflow
It appears that the product has simply become too large. We can instead raise each value to its nth root first and then take the product. Let's also remove all colleges that have missing values or 0's for population.

In [10]:
def geomean(s):
    s = s[s.notna() & (s != 0)]
    power = 1 / len(s)
    s = s ** power
    return s.product()

Let's try this again. Looks like we have are dividing by 0 for those states have no colleges with a valid population.

In [11]:
try:
    college.groupby(['STABBR', 'RELAFFIL']).agg({'UGDS': geomean})
except Exception as e:
    print(type(e), e)

<class 'ZeroDivisionError'> division by zero


Let's return a missing value if there are no valid populations for the state.

In [12]:
def geomean(s):
    s = s[s.notna() & (s != 0)]
    if len(s) == 0:
        return np.NaN
    power = 1 / len(s)
    s = s ** power
    return s.product()

In [13]:
df = college.groupby(['STABBR', 'RELAFFIL']).agg({'UGDS': geomean})
df.head(15)

UGDS
STABBR RELAFFIL             
AK     0         1477.495053
       1           79.628486
AL     0         1051.426021
       1          498.605718
AR     0          446.853645
       1          527.124044
AS     0         1276.000000
AZ     0          574.250943
       1          179.316817
CA     0          618.554204
       1          541.163117
CO     0          516.158987
       1         2388.231594
CT     0          437.256788
       1          988.908112

## Other GroupBy methods similar to `agg`


There are three other methods that work similarly to **`agg`** you will use once you group your DataFrame columns. The following command is the **generic form**. The table below summarizes **`agg`**, **`filter`**, **`transform`** and **`apply`**.

**`df.groupby(['columns', 'to', 'group']).agg/filter/transform/apply(functions to apply)`**

<table>
    <thead>
        <tr>
            <td><b>groupby Method</b></td>
            <td><b>Description</b></td>
            <td><b>Object Passed to Function</b></td>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>agg</td>
            <td>By definition an aggregation means to take many values and reduce them to a single value.<br>
                The agg method must return a single value. <br>
                Can specify a different aggregation for each function.</td>
            <td>Series</td>
        </tr>
        
        <tr>
            <td>filter</td>
            <td>The function must return a boolean. Each group is either kept or discarded.<br> 
                The number of columns always remains the same.</td>
            <td>DataFrame</td>
        </tr>
        
        <tr>
            <td>transform</td>
            <td>The function is applied column by column and keeps the length of the data the same.<br></td>
            <td>Series</td>
        </tr>
        
        <tr>
            <td>apply</td>
            <td>Use only when above methods don't work.<br> 
                   When you want to work with multiple columns within each group.</td>
            <td>DataFrame</td>
        </tr>
    </tbody>
</table>

## Filtering out groups

**`groupby`** objects come with a **`filter`** method that...
1. Scans each group independently
2. Applies a function to each group and returns a boolean value
3. Keeps the group or drops each group based on the boolean value returned from the function
4. The end result is the original DataFrame (same number of columns) with certain groups filtered out

The **`filter`** method accepts a function that returns either True or False for each group. This result is used to filter the original DataFrame.

The **`filter`** method accepts will be a **custom** function that is **`implicitly`** passed a DataFrame of each group.

Anything can happen inside the body of the function passed to **`filter`** but it must return **`True`** or **`False`**. This boolean value determines whether the group is included or is dropped from the final resulting DataFrame.

### Find states with more than 300,000 undergraduate students
To help provide some context, we will use the filter method to find states that have at least 300,000 undergraduate students. The **`filter`** is passed a function that will sum all the undergraduate students for each state. It will take this sum and compare it against the number 300,000 and return **`True`** or **`False`**. Only states that have more than 300,000 students will remain.

None of the values in the DataFrame are mutated. Only rows are dropped with **`filter`**

In [14]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 40

college = pd.read_csv('../../data/college.csv')

In [15]:
# Define function that accepts a dataframe of the current group
# it must return a boolean

def filter_ugds(df):
    return df['UGDS'].sum() > 300000

In [16]:
# use filter
college_filtered = college.groupby('STABBR').filter(filter_ugds)

In [17]:
# see the difference in size
print(college.shape)
print(college_filtered.shape)

(7535, 27)
(4619, 27)


### Pass additional parameters to the filtering function
At first glance the **`filter`** method from above appears to only allow the passed function to contain a single argument. This is not the case. You can pass any number of arguments to the function inside of **`filter`**.

Let's take a look at the **`filter`** docstrings for a moment. Since **`filter`** is a chained method the docstring intelligence tricks (shift + tab + tab, etc...) are not available to us. We can use the help function to output the docstrings into the notebook.

In [18]:
help(college.groupby('STABBR').filter)

Help on method filter in module pandas.core.groupby.groupby:

filter(func, dropna=True, *args, **kwargs) method of pandas.core.groupby.groupby.DataFrameGroupBy instance
    Return a copy of a DataFrame excluding elements from groups that
    do not satisfy the boolean criterion specified by func.
    
    Parameters
    ----------
    f : function
        Function to apply to each subframe. Should return True or False.
    dropna : Drop groups that do not pass the filter. True by default;
        if False, groups that evaluate False are filled with NaNs.
    
    Notes
    -----
    Each subframe is endowed the attribute 'name' in case you need to know
    which group you are working on.
    
    Examples
    --------
    >>> import pandas as pd
    >>> df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
    ...                           'foo', 'bar'],
    ...                    'B' : [1, 2, 3, 4, 5, 6],
    ...                    'C' : [2.0, 5., 8., 1., 2., 9.]})
    >>> grouped = d

### \*args and \*\*kwargs
Notice in the method header the two additional parameters, \*args and \*\*kwargs that not formally mentioned in the parameter list. These two additional parameters will not be explained here but there is a great [stackoverflow post](http://stackoverflow.com/questions/3394835/args-and-kwargs) where you can learn all about them.

### What it means for `filter`
You can pass additional arguments to the function that **`filter`** accepts. This becomes useful when we want to customize the filter based on some parameter.

Let's rebuild the filtering function to accept a **`num_students`** argument that allows more flexibility when determining which states to filter out.

In [19]:
# define new function with additional arguments
def filter_ugds_param(df, num_students):
    return df['UGDS'].sum() > num_students

In [20]:
college_filtered2 = college.groupby('STABBR').filter(filter_ugds_param, num_students=500000)

In [21]:
print(college_filtered2.shape)
print(college_filtered.shape)
print(college.shape)

(3319, 27)
(4619, 27)
(7535, 27)


### Groupby with Series

Pandas Series also have a **`groupby`** method. You might be thinking how it's possible to group and aggregate a single column of data. It becomes possible when you consider the **index** and the many levels that an index can have. Multi-level indexes will be discussed in another notebook.

To make the index meaningful set it to be one of the columns of the DataFrame with the **`set_index`** method.

In [22]:
# create a new dataframe with a more interesting index
college_state_index = college.set_index('STABBR')

college_state_index.head()

,INSTNM,CITY,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
STABBR,,,,,,,,,,,,,,,,,,,,,,,,,,
AL,Alabama A & M University,Normal,1.0,0.0,0.0,0,424.0,420.0,0.0,4206.0,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
AL,University of Alabama at Birmingham,Birmingham,0.0,0.0,0.0,0,570.0,565.0,0.0,11383.0,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
AL,Amridge University,Montgomery,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
AL,University of Alabama in Huntsville,Huntsville,0.0,0.0,0.0,0,595.0,590.0,0.0,5451.0,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
AL,Alabama State University,Montgomery,1.0,0.0,0.0,0,425.0,430.0,0.0,4811.0,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


## Tranforming and not aggregating groups

The most common operation to apply to a group is some kind of aggregation. Getting a single number summary of a group is usually of primary concern. There are however instances where the entire group would like to be transformed with every row kept. The **`transform`** groupby method will apply a (usually) custom function to each column of each group returning data that is the same length as the group. 

One of the most common operations is to standardize data - that is transform a numerical group so that its mean is 0 and standard deviation is 1. This is [done in the documentation](http://pandas.pydata.org/pandas-docs/stable/groupby.html#transformation). 

### Finding the percentage of the total for each group
The transform method implicitly accepts a Series and must return a Series the same length as the original. Here, our transform function takes in the Series of undergrads for each state and divides it by the total and returns this quantity as a Series.

You will need to select the transformation with brackets after the call to **`groupby`**.

In [23]:
def pct_total(s):
    return s / s.sum()

ugds_pct_total = college.groupby('STABBR')['UGDS'].transform(pct_total)
ugds_pct_total.head()

0    0.016939
1    0.045844
2    0.001172
3    0.021953
4    0.019376
Name: UGDS, dtype: float64

In [24]:
# its the same length as the DataFrame
len(ugds_pct_total), len(college)

(7535, 7535)

### Adding a new column with transform
Transform is great if you would like to append a new column to the DataFrame.

In [25]:
college['UGDS_PCT_TOTAL'] = ugds_pct_total

Find the colleges that represent the highest percent of their state

In [26]:
college[['INSTNM', 'STABBR', 'UGDS', 'UGDS_PCT_TOTAL']].sort_values('UGDS_PCT_TOTAL', ascending=False).head(15)

,INSTNM,STABBR,UGDS,UGDS_PCT_TOTAL
4216,University of the Virgin Islands,VI,1971.0,1.000000
4141,Northern Marianas College,MP,1120.0,1.000000
4138,American Samoa Community College,AS,1276.0,1.000000
4214,College of Micronesia-FSM,FM,2344.0,1.000000
4561,College of the Marshall Islands,MH,1078.0,1.000000
4215,Palau Community College,PW,602.0,1.000000
4140,University of Guam,GU,3607.0,0.634812
60,University of Alaska Anchorage,AK,12865.0,0.516004
4137,University of Wyoming,WY,9910.0,0.401410
691,University of Delaware,DE,18222.0,0.384999


# Case Study: Tracking Weight Loss per Month
There are two friends interested in tracking their weight over the course of several months. To provide motivation they decide to wager some money each month. The friend who loses the highest percentage of body weight each month wins that month. Each month is independent from the others so the weight loss percentage resets at the start of the month.

Below is the data that was collected over the course of the bet.

In [27]:
# create fake data
np.random.seed(1234)
all_weight = np.zeros(32)
all_weight[::2] = 300 * np.cumprod(np.random.rand(16) * .05 + .96)
all_weight[1::2] = 200 * np.cumprod(np.random.rand(16) * .05 + .96)
all_weight = np.round(all_weight, 0).astype(int)


df_weight = pd.DataFrame({'Name':['Bob', 'Amy'] * 16, 
                          'Month': ('Jan ' * 8 + ' Feb' * 8 + ' Mar' * 8 + ' Apr' * 8).split(),
                          'Week' : (['Week 1'] * 2 + ['Week 2'] * 2 + ['Week 3'] * 2 + ['Week 4'] * 2) * 4,
                          'Weight':all_weight},
                        columns=['Name', 'Month', 'Week', 'Weight'])

df_weight

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,291
1,Amy,Jan,Week 1,197
2,Bob,Jan,Week 2,288
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,283
5,Amy,Jan,Week 3,189
6,Bob,Jan,Week 4,283
7,Amy,Jan,Week 4,190
8,Bob,Feb,Week 1,283
9,Amy,Feb,Week 1,186


### Transform
A new bet is begun at the start of each month and the winner for each month is declared by percentage weight loss achieved from week 1 to week 4. Percentage weight loss resets to zero at the beginning of each month.

We need to find a way to track the percentage weight loss within each month for each person. Since each week will have a weight loss percentage, no aggregation is performed and instead a new value is needed for each row. This situation calls for **`transform`**.

The default behavior of transform is to apply the same function (given as an argument) to each non-grouped column of the DataFrame. Since columns are Series, it is a Series that is passed to the transformation function.

In [28]:
# define a custom function that takes a Series and find the percentage loss from the first week
# is passed a Series

def find_perc_loss(s):
    return (s - s.iloc[0]) / s.iloc[0]

In [29]:
# group by name and month
# apply transformation
# and only transform weight column
df_weight['percent_month_loss'] = df_weight.groupby(['Name', 'Month'])['Weight'].transform(find_perc_loss) 

# view first two months. Notice that percent loss resets to 0
df_weight.head(16)

,Name,Month,Week,Weight,percent_month_loss
0,Bob,Jan,Week 1,291,0.000000
1,Amy,Jan,Week 1,197,0.000000
2,Bob,Jan,Week 2,288,-0.010309
3,Amy,Jan,Week 2,189,-0.040609
4,Bob,Jan,Week 3,283,-0.027491
5,Amy,Jan,Week 3,189,-0.040609
6,Bob,Jan,Week 4,283,-0.027491
7,Amy,Jan,Week 4,190,-0.035533
8,Bob,Feb,Week 1,283,0.000000
9,Amy,Feb,Week 1,186,0.000000


In [30]:
# it might be easier to read if sorted.
# note that Month is not sorted lexicographically and not by calendar
df_weight_final = df_weight.sort_values(['Name', 'Month', 'Week'])

df_weight_final

,Name,Month,Week,Weight,percent_month_loss
25,Amy,Apr,Week 1,166,0.000000
27,Amy,Apr,Week 2,164,-0.012048
29,Amy,Apr,Week 3,164,-0.012048
31,Amy,Apr,Week 4,161,-0.030120
9,Amy,Feb,Week 1,186,0.000000
11,Amy,Feb,Week 2,184,-0.010753
13,Amy,Feb,Week 3,177,-0.048387
15,Amy,Feb,Week 4,173,-0.069892
1,Amy,Jan,Week 1,197,0.000000
3,Amy,Jan,Week 2,189,-0.040609


### Finding a winner

It's possible to manually find a winner of each month by comparing each person's week 4. For instance, Amy lost 7.9% of her body weight in April compared to Bob's 6.6% and won that month.

This is very tedious and since we have Pandas would be ridiculous to do by hand. We need to reshape the data in such a manner that each person's week 4 is easily comparable. There are many ways to reshape data. **`pivot`** and **`pivot_table`** DataFrame methods allow you to convert **long** formatted data into **wide** formatted data. This converts column values into column names. More will be said on this later.

First the above DataFrame will be filtered for only week 4 and then pivoted to make the comparison easy.

In [31]:
df_weight_week4 = df_weight_final[df_weight_final['Week'] == 'Week 4']

df_weight_week4

,Name,Month,Week,Weight,percent_month_loss
31,Amy,Apr,Week 4,161,-0.030120
15,Amy,Feb,Week 4,173,-0.069892
7,Amy,Jan,Week 4,190,-0.035533
23,Amy,Mar,Week 4,170,-0.028571
30,Bob,Apr,Week 4,250,-0.038462
14,Bob,Feb,Week 4,268,-0.053004
6,Bob,Jan,Week 4,283,-0.027491
22,Bob,Mar,Week 4,261,-0.033333


In [32]:
# use pivot to move the Name column
df_weight_winner = df_weight_week4.pivot(index='Month', columns='Name', values='percent_month_loss')

df_weight_winner

Name,Amy,Bob
Month,,
Apr,-0.030120,-0.038462
Feb,-0.069892,-0.053004
Jan,-0.035533,-0.027491
Mar,-0.028571,-0.033333


### Column for winner with np.where

Now that the winner is much more easily seen with the new reshaped data, a final step of creating a column with the winner's name can be made with the numpy **`where`** function. **`np.where`** works by an array of boolean values and returns an array consisting of the second argument wherever the array is True and the third argument False.

In [33]:
# a trivial example
# Return 'Yes' when True and 'No' when False

np.where([True, False, False, True, False, True], 'Yes', 'No')

array(['Yes', 'No', 'No', 'Yes', 'No', 'Yes'], dtype='<U3')

In [34]:
# make the winner Amy when her weight loss is more than Bob's and vice versa using np.where
df_weight_winner['Winner'] = np.where(df_weight_winner['Amy'] < df_weight_winner['Bob'], 'Amy', 'Bob')

df_weight_winner

Name,Amy,Bob,Winner
Month,,,
Apr,-0.030120,-0.038462,Bob
Feb,-0.069892,-0.053004,Amy
Jan,-0.035533,-0.027491,Amy
Mar,-0.028571,-0.033333,Bob


In [35]:
# Get the winner
# If there happen to be lots of months of data

df_weight_winner['Winner'].value_counts()

Bob    2
Amy    2
Name: Winner, dtype: int64

# Exercises
We will be working with the city of Houston dataset for the questions in this notebook. Run the following command before attempting the problems. Solutions are below.

In [36]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 40
employee = pd.read_csv('../../data/employee.csv')
employee.head()

,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE
0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13
1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18
2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03
3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25
4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22


## Problem 1
<span  style="color:green; font-size:16px">What are the 5 least common departments?</span>

In [37]:
# your code here

### Problem 2
<span  style="color:green; font-size:16px">Filter out departments with less than 50 occurences and save it to **`employee_filter`**. Then test your code by outputing the frequencies of all the remaining departments. </span>

In [38]:
# your code here

### Problem 3
<span  style="color:green; font-size:16px">Filter out departments from the original **`employee`** DataFrame with average salaries less than $70,000 and save it to **`employee_filter_salary`**. Then test your code by outputing the average salaries for the remaining departments.</span>

In [39]:
# your code here

### Problem 4
<span  style="color:green; font-size:16px">Filter *`for`* those departments from the original **`employee`** DataFrame with average salaries of at least 65,000 or having at least 25 unique position titles. Save result to **`employee_more`**</span>

In [40]:
# your code here

### Problem 5: Advanced
<span  style="color:green; font-size:16px">Find a way to do problem 4 without using the **`filter`** method. Make clever use of aggregate groupby and boolean logic</span>

In [41]:
# your code here

### Problem 6
<span  style="color:green; font-size:16px"> Create a column **`is_max`** that is equal to 1 if the base salary is currently the max base salary (out of all previous rows) for that department and 0 otherwise. See sample data below.</span>

In [42]:
# your code here

In [43]:
# Return a dataframe that looks like this
# 
# DEPARTMENT    BASE_SALARY  is_max  
# Library           160         1
# Police            150         1
# Library           170         1
# Police             95         0
# Police            140         0
# Library            80         0
# Police            189         1
# 

### Problem 7: Advanced
<span  style="color:green; font-size:16px"> Programatically Find the 10th occurence of 0 for **`is_max`** and return a DataFrame that ends after the tenth occurence.</span>

In [44]:
# your code here

### Problem 8
<span  style="color:green; font-size:16px"> Write a function that accepts a single argument that will filter **`employee1`** for a specific department where **`is_max`** is 1. Test your function with departments like 'Library' and 'Public Works & Engineering-PWE'.</span>

In [45]:
# your code here

### Problem 9
<span  style="color:green; font-size:16px">A good skill to have is to ask a difficult question for yourself and then answer it. Ask yourself a question that involes grouping and answer it.</span>

In [46]:
# your code here

# Solutions

## Problem 1
<span  style="color:green; font-size:16px">What are the 5 least common departments?</span>

In [47]:
employee.DEPARTMENT.value_counts().tail()

Houston Information Tech Svcs    9
Planning & Development           7
City Controller's Office         5
Mayor's Office                   5
Convention and Entertainment     1
Name: DEPARTMENT, dtype: int64

### Problem 2
<span  style="color:green; font-size:16px">Filter out departments with less than 50 occurences and save it to **`employee_filter`**. Then test your code by outputing the frequencies of all the remaining departments. </span>

In [48]:
employee_filter = employee.groupby('DEPARTMENT').filter(lambda df: len(df) >= 50)

employee_filter.DEPARTMENT.value_counts()

Houston Police Department-HPD     638
Houston Fire Department (HFD)     384
Public Works & Engineering-PWE    343
Health & Human Services           110
Houston Airport System (HAS)      106
Parks & Recreation                 74
Name: DEPARTMENT, dtype: int64

### Problem 3
<span  style="color:green; font-size:16px">Filter out departments from the original **`employee`** DataFrame with average salaries less than $70,000 and save it to **`employee_filter_salary`**. Then test your code by outputing the average salaries for the remaining departments.</span>

In [49]:
employee_filter_salary = employee.groupby('DEPARTMENT').filter(lambda df: df['BASE_SALARY'].mean() >= 70000)

In [50]:
# added astype(int) to remove decimals
employee_filter_salary.groupby('DEPARTMENT')['BASE_SALARY'].mean().astype(int)

DEPARTMENT
Finance                           79650
Houston Information Tech Svcs     76112
Legal Department                 104959
Mayor's Office                    86489
Name: BASE_SALARY, dtype: int64

### Problem 4
<span  style="color:green; font-size:16px">Filter *`for`* those departments from the original **`employee`** DataFrame with average salaries of at least 65,000 or having at least 25 unique position titles. Save result to **`employee_more`**</span>

In [51]:
def more(df):
    return (df['BASE_SALARY'].mean() > 65000) | (df['POSITION_TITLE'].nunique() >= 25)

employee_more = employee.groupby('DEPARTMENT').filter(more)

In [52]:
employee_more.shape

(1696, 9)

### Problem 5: Advanced
<span  style="color:green; font-size:16px">Find a way to do problem 4 without using the **`filter`** method. Make clever use of aggregate groupby and boolean logic</span>

In [53]:
# do aggregations for each boolean piece separately
salary_grp = employee.groupby('DEPARTMENT')['BASE_SALARY'].mean()
uniq_grp = employee.groupby('DEPARTMENT')['POSITION_TITLE'].nunique()

salary_grp.head()

DEPARTMENT
Admn. & Regulatory Affairs      50890.551724
City Controller's Office        55711.600000
City Council                    59089.222222
Convention and Entertainment    38397.000000
Dept of Neighborhoods (DON)     47092.882353
Name: BASE_SALARY, dtype: float64

In [54]:
uniq_grp.head()

DEPARTMENT
Admn. & Regulatory Affairs      19
City Controller's Office         3
City Council                     7
Convention and Entertainment     1
Dept of Neighborhoods (DON)     14
Name: POSITION_TITLE, dtype: int64

In [55]:
# create boolean criteria

deps = (salary_grp > 65000) | (uniq_grp >= 25)

deps

DEPARTMENT
Admn. & Regulatory Affairs        False
City Controller's Office          False
City Council                      False
Convention and Entertainment      False
Dept of Neighborhoods (DON)       False
Finance                            True
Fleet Management Department       False
General Services Department       False
Health & Human Services            True
Housing and Community Devp.       False
Houston Airport System (HAS)       True
Houston Emergency Center (HEC)    False
Houston Fire Department (HFD)      True
Houston Information Tech Svcs      True
Houston Police Department-HPD      True
Human Resources Dept.             False
Legal Department                   True
Library                           False
Mayor's Office                     True
Municipal Courts Department       False
Parks & Recreation                 True
Planning & Development            False
Public Works & Engineering-PWE     True
Solid Waste Management            False
dtype: bool

In [56]:
# filter Series with itself and grab index values
deps_true = deps[deps].index.values

deps_true

array(['Finance', 'Health & Human Services',
       'Houston Airport System (HAS)', 'Houston Fire Department (HFD)',
       'Houston Information Tech Svcs', 'Houston Police Department-HPD',
       'Legal Department', "Mayor's Office", 'Parks & Recreation',
       'Public Works & Engineering-PWE'], dtype=object)

In [57]:
employee_more_check = employee[employee.DEPARTMENT.isin(deps_true)]

In [58]:
# can check equality of dataframes with equals method
employee_more.equals(employee_more_check)

True

### Problem 6
<span  style="color:green; font-size:16px"> Create a column **`is_max`** that is equal to 1 if the base salary is currently the max base salary (out of all previous rows) for that department and 0 otherwise. See sample data below.</span>

In [59]:
employee1 = employee[['DEPARTMENT', 'BASE_SALARY']].copy()
employee1['is_max'] = employee1.groupby('DEPARTMENT')['BASE_SALARY'].transform(lambda x: x == x.cummax())

In [60]:
employee1.head(20)

,DEPARTMENT,BASE_SALARY,is_max
0,Municipal Courts Department,121862.0,True
1,Library,26125.0,True
2,Houston Police Department-HPD,45279.0,True
3,Houston Fire Department (HFD),63166.0,True
4,General Services Department,56347.0,True
5,Houston Police Department-HPD,66614.0,True
6,Public Works & Engineering-PWE,71680.0,True
7,Houston Airport System (HAS),42390.0,True
8,Public Works & Engineering-PWE,107962.0,True
9,Houston Airport System (HAS),44616.0,True


### Problem 7: Advanced
<span  style="color:green; font-size:16px"> Programatically Find the 10th occurence of 0 for **`is_max`** and return a DataFrame that ends after the tenth occurence.</span>

In [61]:
employee1['occur'] = employee1.groupby('is_max').cumcount()
employee1.head(20)

,DEPARTMENT,BASE_SALARY,is_max,occur
0,Municipal Courts Department,121862.0,True,0
1,Library,26125.0,True,1
2,Houston Police Department-HPD,45279.0,True,2
3,Houston Fire Department (HFD),63166.0,True,3
4,General Services Department,56347.0,True,4
5,Houston Police Department-HPD,66614.0,True,5
6,Public Works & Engineering-PWE,71680.0,True,6
7,Houston Airport System (HAS),42390.0,True,7
8,Public Works & Engineering-PWE,107962.0,True,8
9,Houston Airport System (HAS),44616.0,True,9


In [62]:
criteria = (employee1['is_max'] == 0) & (employee1['occur'] == 9)
e2 = employee1[criteria]
e2

,DEPARTMENT,BASE_SALARY,is_max,occur
22,Public Works & Engineering-PWE,43264.0,False,9


In [63]:
idx10 = e2.index.values[0]
idx10

22

In [64]:
employee1.loc[:idx10]

,DEPARTMENT,BASE_SALARY,is_max,occur
0,Municipal Courts Department,121862.0,True,0
1,Library,26125.0,True,1
2,Houston Police Department-HPD,45279.0,True,2
3,Houston Fire Department (HFD),63166.0,True,3
4,General Services Department,56347.0,True,4
5,Houston Police Department-HPD,66614.0,True,5
6,Public Works & Engineering-PWE,71680.0,True,6
7,Houston Airport System (HAS),42390.0,True,7
8,Public Works & Engineering-PWE,107962.0,True,8
9,Houston Airport System (HAS),44616.0,True,9


### Problem 8
<span  style="color:green; font-size:16px"> Write a function that accepts a single argument that will filter **`employee1`** for a specific department where **`is_max`** is 1. Test your function with departments like 'Library' and 'Public Works & Engineering-PWE'.</span>

In [65]:
def filter_dep(dep):
    criteria = (employee1.DEPARTMENT == dep) & (employee1.is_max == 1)
    return employee1[criteria]

In [66]:
filter_dep('Library')

,DEPARTMENT,BASE_SALARY,is_max,occur
1,Library,26125.0,True,1
280,Library,31034.0,True,49
298,Library,38563.0,True,51
308,Library,49317.0,True,53
412,Library,79302.0,True,61
1165,Library,107763.0,True,78


In [67]:
filter_dep('Public Works & Engineering-PWE')

,DEPARTMENT,BASE_SALARY,is_max,occur
6,Public Works & Engineering-PWE,71680.0,True,6
8,Public Works & Engineering-PWE,107962.0,True,8
186,Public Works & Engineering-PWE,110881.0,True,39
297,Public Works & Engineering-PWE,141948.0,True,50
1067,Public Works & Engineering-PWE,146141.0,True,77
1232,Public Works & Engineering-PWE,178331.0,True,80


### Problem 9
<span  style="color:green; font-size:16px">A good skill to have is to ask a difficult question for yourself and then answer it. Ask yourself a question that involes grouping and answer it.</span>

In [68]:
# fill in your answer here